In [17]:
import fitz
import openai

In [45]:
import fitz
import openai
from nltk.tokenize import sent_tokenize
from io import StringIO
import json

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

openai.api_key = ''

def read_pdf(filename):
  context = ''
  
  # Open the PDF file
  with fitz.open(filename) as pdf_file:
  
    # Get the number of pages in the PDF file
    num_pages = pdf_file.page_count
    
    # Loop through each page in the PDF file
    for page_num in range(num_pages):
      
      # Get the current page
      page = pdf_file[page_num]
      
      # Get the text from the current page
      page_text = page.get_text()
      
      # Append the text to context
      context += page_text
  return context

def split_text(text, chunk_size=5000):
  """
  Splits the given text into chunks of approximately the specified chunk size.
  
  Args:
  text (str): The text to split.
  
  chunk_size (int): The desired size of each chunk (in characters).
  
  Returns:
  List[str]: A list of chunks, each of approximately the specified chunk size.
  """
  
  chunks = []
  current_chunk = StringIO()
  current_size = 0
  sentences = sent_tokenize(text)
  for sentence in sentences:
    sentence_size = len(sentence)
    if sentence_size > chunk_size:
      while sentence_size > chunk_size:
        chunk = sentence[:chunk_size]
        chunks.append(chunk)
        sentence = sentence[chunk_size:]
        sentence_size -= chunk_size
        current_chunk = StringIO()
        current_size = 0
    if current_size + sentence_size < chunk_size:
      current_chunk.write(sentence)
      current_size += sentence_size
    else:
      chunks.append(current_chunk.getvalue())
      current_chunk = StringIO()
      current_chunk.write(sentence)
      current_size = sentence_size
  if current_chunk:
     chunks.append(current_chunk.getvalue())
  return chunks
  

def gpt3_completion(prompt, engine='text-davinci-003', temp=0.5, top_p=0.3, tokens=1000):
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    try:
        response = openai.Completion.create(
  engine=engine,
  prompt=prompt,
  temperature=temp,
  top_p=top_p,
  max_tokens=tokens
  )
        return response.choices[0].text.strip()
    except Exception as oops:
        return "GPT-3 error: %s" % oops


def summrize(documnet):
  
  # Calling the split function to split text
  chunks = split_text(documnet)
  
  summaries = []
  for chunk in chunks:
    prompt = 'Please summarize the following documnet include the name of the Professor, course topics and limit to 300 tokens  \n'
    summary = gpt3_completion(prompt + chunk)

    if summary.startswith('GPT-3 error:'):
        continue

    summaries.append(summary)
  return ''.join(summaries)


#read the pdf file
document = read_pdf('./pdfs/2023 Spring_ISM 6562_901_Big Data for Business Applications_Smith, Tim.pdf')

# Call the summrize function with the document as input
summrize(document)

'BOOK \nBig Data Analytics with Spark: A Practitioner\'s Guide to Using Spark for Large Scale Data Analysis. \n \n \nIn this course, Dr. Tim Smith will cover topics on Big Data for Business Applications. The course will cover data management and data analytics for businesses using various big data technologies such as No-SQL database, distributed file system, Map-Reduce, and Spark. The course will be delivered face to face in class at BSN 230 and the minimum technical skills and requirements can be found at https://www.usf.edu/it/remote/requirements-for-students.aspx. Virtual office hours will be held on Monday 11:30 am to 12:30 pm using Microsoft Teams. The course materials include Frank Kane\'s Taming Big Data with Apache Spark and Python, Ullman Mining of Massive Datasets, PySpark Cookbook, and Big Data Analytics with Spark.This course is taught by Professor Ivan Marin, Ankit Shukla, and Sarang VK. It covers topics such as Big Data Application, No-SQL Database Basics and concepts, C

In [46]:
# Specify input and output folders
pdf_folder = 'pdfs'
word_folder = 'pdf_word'

# Ensure the output folder exists, create it if not
os.makedirs(word_folder, exist_ok=True)

# List all PDF files in the input folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

In [51]:
pdf_files

['2023 Spring _ MAN 3301-021_Human Resource Management_online_Boyd, Terry A..pdf',
 '2023 Spring _GEB 4890-903_Strategic Management and Decision Making_, Papp, Timothy.pdf',
 '2023 Spring _ISM 6124_-521_Advanced Systems Analysis & Design_Unhelkar, Bhuvan.pdf',
 '2023 Spring _MAN 4930-522_Business Analysis_Unhelkar, Bhuvan.pdf',
 '2023 Spring-ISM 6562-001_Big Data for Business Applications_Dutta, Kaushik.pdf',
 '2023 Spring-QMB- 6358-901_Data Analytics for Business_Garimella, Kiran.pdf',
 '2023 Spring_ ISM 3011_691_Information Systems in Organizations_Burleson, Rea.pdf',
 '2023 Spring_ ISM 6419-001_ Data Visualization for Story Telling_Reichgelt, Han.pdf',
 '2023 Spring_ MAN 3240- 791_Organizational Behavior Analysis_ Solomon, Nichole.pdf',
 '2023 Spring_CGS 2100-020-021-Computers in Business-Walpole, Dennis.pdf',
 '2023 Spring_CGS 2100-691-Computing in Business_Pelzer, Jeff.pdf',
 '2023 Spring_CGS 2100_521-Computers in Business_Casado, Mara.pdf',
 '2023 Spring_CGS 2100_522_Computers in

In [47]:
import os
from PyPDF2 import PdfReader
from docx import Document

In [63]:
for i in pdf_files:
    doc = Document()
    #read the pdf file
    document = read_pdf('./pdfs/'+i)
    text = summrize(document)
    doc.add_paragraph(text)
    word_file = i.replace('.pdf', '.docx')
    word_path = os.path.join(word_folder, word_file)
    doc.save(word_path)
    print(f'Converted {pdf_file} to {word_file}')

Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring _ MAN 3301-021_Human Resource Management_online_Boyd, Terry A..docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring _GEB 4890-903_Strategic Management and Decision Making_, Papp, Timothy.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring _ISM 6124_-521_Advanced Systems Analysis & Design_Unhelkar, Bhuvan.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring _MAN 4930-522_Business Analysis_Unhelkar, Bhuvan.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring-ISM 6562-001_Big Data for Business Applications_Dutta, Kaushik.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring-QMB- 6358-901_Data Analytics for Business_Garimella, Kiran.docx
Conv

Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6124-901_Advanced Systems Analysis & Design_Green, Marcus.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6124-903_Advanced Systems Analysis & Design_Green, Marcus.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6136-001_Data Mining_Ebrahim,Mohammadreza.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6136-350-020_Data Mining_Garimella, Kiran.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6137-001-902-Statistical Data Mining_Bhattacherjee, Anol.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_ISM 6137-901-Statistical Data Mining_Zantedeschi, Daniel.docx
Converted 2023 Spring_ISM 3113 -020-System

Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4282-521_Organizational Assessment_Cooley_(Dug) Mack.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4282-901_Organizational Assessment_Michaels, Charles.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4283-001_Organizational Assessment_Michaels, Charles.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4402-901 & MAN 6406-901_Employment Law for Business_Sheslow,William.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4402-902_Employment Laws_Papp,Timothy.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Spring_MAN 4430 -Negotiation and Administration of Labor Agreements_Karlesky, Matthew .docx
Converted 2

Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 4220 - Business Data Communications - Giti Javidi.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 4300 - Managing Information Resources - Weimar Ardila Rueda.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 6021.350 - Management Information Systems - McGrath, Quintin.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 6218 - Advanced Database Management - Han Reichgelt.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 6577.320 - Decision Processes for Business Continuity and Disaster Recovery (BCDR) -Marcus Green.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to 2023 Summer - ISM 6940 - ISM 6940 Practicum

Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to ISM 6930 - Ma.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to ISM 6930 - Vreede.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 3025 - Karlins.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 3025 - Rolando.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 30250.21 - Dozier.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 4402 - Papp.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 6055 - Donahue.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN 6930 - Weiss.docx
Converted 2023 Spring_ISM 3113 -020-Systems Analysis & Desigs_ Hyman, Harvey H..pdf to MAN6055 - Weiss.docx
Converted 20